In [37]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs
import re

import json
import pickle

In [38]:
headers = {'User-Agent':'Mozilla/5.0'}
base_url = "https://www.tripadvisor.com.tr/Restaurants-g293969-Turkey.html?page="

In [39]:
def getAndParse(url,headers):
    response=requests.get(url,headers,timeout=(700000,700000))
    soup=bs(response.text)
    return soup

In [4]:
response=requests.get("https://www.tripadvisor.com/Restaurants-g186338-London_England.html",headers)
s=bs(response.text)
s.findAll(class_="pageNumbers") # sayfa düzeni

[<div class="pageNumbers">
 <span class="pageNum current cx_brand_refresh_phase2" data-offset="0" data-page-number="1" onclick="ta.trackEventOnPage('STANDARD_PAGINATION', 'curpage', '1', 0)">1</span>
 <a class="pageNum taLnk" data-offset="30" data-page-number="2" href="/Restaurants-g186338-oa30-London_England.html#EATERY_LIST_CONTENTS" onclick="      require('common/Radio')('restaurant-filters').emit('paginate', this.getAttribute('data-offset'));; ta.trackEventOnPage('STANDARD_PAGINATION', 'page', '2', 0); return false;
   ">
 2
 </a>
 <a class="pageNum taLnk" data-offset="60" data-page-number="3" href="/Restaurants-g186338-oa60-London_England.html#EATERY_LIST_CONTENTS" onclick="      require('common/Radio')('restaurant-filters').emit('paginate', this.getAttribute('data-offset'));; ta.trackEventOnPage('STANDARD_PAGINATION', 'page', '3', 0); return false;
   ">
 3
 </a>
 <a class="pageNum taLnk" data-offset="90" data-page-number="4" href="/Restaurants-g186338-oa90-London_England.html#EA

In [28]:
page_links = [] # first page
page_links.append("https://www.tripadvisor.com/Restaurants-g186338-London_England.html#EATERY_LIST_CONTENTS")  #1650

In [48]:
page_links = []
for i in range(30,19620,30):
    page_links.append("https://www.tripadvisor.com.tr//Restaurants-g186338-oa"+str(i)+"-London_England.html#EATERY_LIST_CONTENTS")

In [49]:
cafe_names = []
cafe_links = []
cafe_tips = []

for page in page_links: # bütün restaurant linkleri burdan
    soup = getAndParse(page,headers)
    cafe_names+=[x.text for x in soup.findAll("div",class_="wQjYiB7z")]
    cafe_links+=['https://www.tripadvisor.com.tr'+x.a.get('href') for x in soup.findAll("div",class_="wQjYiB7z")]
    # cafe_tips+=[x.text for x in soup.findAll("span",class_="_1p0FLy4t")]
    

In [50]:
import pandas as pd
df = pd.DataFrame()
df["cafe_names"] = cafe_names
df["cafe_links"] = cafe_links

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 665 entries, 0 to 664
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cafe_names  665 non-null    object
 1   cafe_links  665 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


In [52]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 593 entries, 0 to 664
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   cafe_names  593 non-null    object
 1   cafe_links  593 non-null    object
dtypes: object(2)
memory usage: 13.9+ KB


In [54]:
df = df.reset_index()
df = df.drop(columns=["index"])
df.head()

,cafe_names,cafe_links
0,Pane Cunzato - Pizza & more,https://www.tripadvisor.com.tr/Restaurant_Revi...
1,1921. Wahaca Kentish Town,https://www.tripadvisor.com.tr/Restaurant_Revi...
2,1922. The Phoenix,https://www.tripadvisor.com.tr/Restaurant_Revi...
3,1923. Pizza East Portobello,https://www.tripadvisor.com.tr/Restaurant_Revi...
4,1924. Terroirs,https://www.tripadvisor.com.tr/Restaurant_Revi...


In [65]:
review_links = [] # ***  -----
for i in range(len(df["cafe_links"])):
    soup = getAndParse(df["cafe_links"][i],headers)
    review_link = ["https://www.tripadvisor.com/"+x.get("href") for x in soup.findAll("a",class_=re.compile("pageNum"))]
    review_links.append(dict([
        ('name', df["cafe_names"][i]),
        ('links', review_link)
        ]))
with open('review_link_night2.pickle', 'wb') as handle:
    pickle.dump(review_links, handle)

In [66]:
# Not : linkten "tr" leri çıkart !
new_df = pd.DataFrame(data=review_links)
new_df = new_df[["name","links"]]
new_df

,name,links
0,Pane Cunzato - Pizza & more,[https://www.tripadvisor.com//Restaurant_Revie...
1,1921. Wahaca Kentish Town,[https://www.tripadvisor.com//Restaurant_Revie...
2,1922. The Phoenix,[https://www.tripadvisor.com//Restaurant_Revie...
3,1923. Pizza East Portobello,[https://www.tripadvisor.com//Restaurant_Revie...
4,1924. Terroirs,[https://www.tripadvisor.com//Restaurant_Revie...
...,...,...
588,2457. Bianco43 - Trafalgar Square,[https://www.tripadvisor.com//Restaurant_Revie...
589,2458. North London Tavern,[https://www.tripadvisor.com//Restaurant_Revie...
590,2459. Butchers Hook & Cleaver,[https://www.tripadvisor.com//Restaurant_Revie...
591,2460. The Southampton Arms,[https://www.tripadvisor.com//Restaurant_Revie...


In [67]:
new = (new_df.groupby('name',as_index=False)
          .agg({'links':(lambda x: x)})
          .rename(columns={'links':'all_links'}))
l = new.all_links.to_list()
import numpy as np
r = []
for i in l:
    if(i==[]):
        r.append(np.NaN)
    else:
        r.append(i)

new.all_links = r
new = new.dropna()

<ipython-input-67-897fdb4f4e83>:8: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if(i==[]):


In [68]:
new = new.reset_index()
#new_dic= dict(new)

#review_links = []
#for i in range(len(new.all_links)):
#    review_links.append(dict([
#('name', new["name"][i]),
#('links', new["all_links"][i])]))
    

#review_links

review_links = []
for i in range(len(new.all_links)):
    for x in range(len(new.all_links[i])):
        
        review_links.append(dict([
('name', new["name"][i]),
('links', new["all_links"][i][x])]))
    

review_links

[{'name': '1921. Wahaca Kentish Town',
  'links': 'https://www.tripadvisor.com//Restaurant_Review-g186338-d8263607-Reviews-Wahaca_Kentish_Town-London_England.html'},
 {'name': '1921. Wahaca Kentish Town',
  'links': 'https://www.tripadvisor.com//Restaurant_Review-g186338-d8263607-Reviews-or10-Wahaca_Kentish_Town-London_England.html'},
 {'name': '1921. Wahaca Kentish Town',
  'links': 'https://www.tripadvisor.com//Restaurant_Review-g186338-d8263607-Reviews-or20-Wahaca_Kentish_Town-London_England.html'},
 {'name': '1921. Wahaca Kentish Town',
  'links': 'https://www.tripadvisor.com//Restaurant_Review-g186338-d8263607-Reviews-or30-Wahaca_Kentish_Town-London_England.html'},
 {'name': '1921. Wahaca Kentish Town',
  'links': 'https://www.tripadvisor.com//Restaurant_Review-g186338-d8263607-Reviews-or40-Wahaca_Kentish_Town-London_England.html'},
 {'name': '1922. The Phoenix',
  'links': 'https://www.tripadvisor.com//Restaurant_Review-g186338-d2700557-Reviews-The_Phoenix-London_England.html'},


In [69]:
new_df2 = pd.DataFrame(review_links)

l = new_df2["links"].to_list()

import numpy as np
r = []
for i in l:
    if(i=="https://www.tripadvisor.com/"):
        r.append(np.NaN)
    else:
        r.append(i)

new_df2["links"] = r
new_df2 = new_df2.dropna()
new_df2 = new_df2.reset_index()

In [70]:
review_links3 = []
for i in range(len(new_df2.links)):
    review_links3.append(dict([
('name', new_df2["name"][i]),
('links', new_df2["links"][i])]))

In [ ]:
import time
import random
sleepTimes = [0.75, 0.5, 0.1,1,0.82,1.2,0.97]


name = []
rating = []
comments = []
summary = []
info = []
lin = []
users = []
i_d = []
# bunu al bu çok iyi yıldızzz çok yıldızzzz len(review_links2)
for r in range(len(review_links3)):    
 #   for links in review_links[r]["links"]:
    soup = getAndParse(review_links3[r]["links"],headers)
    time.sleep(random.choice(sleepTimes))
    comments+=([text.findNext().text for text in soup.findAll("span",class_="noQuotes")])
    users+=[text.text for text in soup.findAll("div",class_="info_text pointer_cursor")]
    for i in range(len(soup.findAll("div",class_="ui_column is-9"))): 
        s = soup.findAll("div",class_="ui_column is-9")
        rating+=([re.findall(r"(?: bubble_)+(\d{,2})",str(r)) for r in s[i].findAll("span",class_=re.compile("ui_bubble_rating bubble_"))]) 
        for x in range(len(s[i].findAll("span",class_=re.compile("ui_bubble_rating bubble_")))):
            name.append([(review_links3[r]["name"])])
            info.append([text.text for text in soup.findAll("div",class_= "_2170bBgV")])
            summary.append([text.text for text in soup.findAll("div",class_="_1lSTB9ov")])
            lin.append([text.text for text in soup.findAll("a",class_="_2wKz--mA _27M8V6YV")])
            i_d.append(x)  

In [353]:
last_df = pd.DataFrame()
last_df["name"] = name
last_df["id"] = i_d
last_df["rating"] = rating
last_df["comments"] = comments

last_df["users"] = users
last_df["summary"] = summary
last_df["info"] = info
last_df["loc"] = lin
            
last_df.rating = last_df.rating.apply(lambda x : x[0])
last_df["name"] = last_df["name"].apply(lambda x : x[0])
#last_df["loc"] = last_df["loc"].apply(lambda x : x[0])

l = last_df["summary"].to_list()


import numpy as np
r = []
for i in l:
    if(i==[]):
        r.append([np.NaN])
    else:
        r.append(i)

last_df["summary"] = r

m = last_df["info"].to_list()


import numpy as np
k = []
for i in m:
    if(i==[]):
        k.append(np.NaN)
    else:
        k.append(i)

last_df["info"] = k
            
last_df.summary = last_df.summary.apply(lambda x : x[0])

In [123]:
last_df = pd.DataFrame()
last_df["name"] = name
last_df["id"] = i_d
last_df["rating"] = rating
last_df["comments"] = comments
last_df["users"] = users
last_df["summary"] = summary
last_df["info"] = info
last_df["loc"] = lin
            
last_df.rating = last_df.rating.apply(lambda x : x[0])
last_df["name"] = last_df["name"].apply(lambda x : x[0])
#last_df["loc"] = last_df["loc"].apply(lambda x : x[0])

l = last_df["summary"].to_list()


import numpy as np
r = []
for i in l:
    if(i==[]):
        r.append([np.NaN])
    else:
        r.append(i)

last_df["summary"] = r

m = last_df["info"].to_list()


import numpy as np
k = []
for i in m:
    if(i==[]):
        k.append(np.NaN)
    else:
        k.append(i)

last_df["info"] = k
            
last_df.summary = last_df.summary.apply(lambda x : x[0])

In [354]:
last_df.tail()

,name,id,rating,comments,users,summary,info,loc
10625,1743. St. John,0,30,A private dining event with a suckling pig as ...,splendidc2018,Fergus Henderson's Michelin-starred home of No...,"[TRY 349 - TRY 349, Gluten Free Options, Lunch...","[26 St. John Street, London EC1M 4AY England, ..."
10626,1743. St. John,0,40,Have been coming to St John for over 15 years ...,aka_fitzy,Fergus Henderson's Michelin-starred home of No...,"[TRY 349 - TRY 349, Gluten Free Options, Lunch...","[26 St. John Street, London EC1M 4AY England, ..."
10627,1743. St. John,0,20,I am a great fan of offal and own both Nose to...,Z9195TLfrankh,Fergus Henderson's Michelin-starred home of No...,"[TRY 349 - TRY 349, Gluten Free Options, Lunch...","[26 St. John Street, London EC1M 4AY England, ..."
10628,1743. St. John,0,10,Terrible. Why?\n1) 83GBP / person\n2) low qual...,Lulu66Lulu,Fergus Henderson's Michelin-starred home of No...,"[TRY 349 - TRY 349, Gluten Free Options, Lunch...","[26 St. John Street, London EC1M 4AY England, ..."
10629,1743. St. John,0,30,This meal was fine. I don't think it's a must-...,Henry Y,Fergus Henderson's Michelin-starred home of No...,"[TRY 349 - TRY 349, Gluten Free Options, Lunch...","[26 St. John Street, London EC1M 4AY England, ..."


In [308]:
last_df.tail()

,name,id,rating,comments,users,summary,info,loc
13425,1631. Busaba Soho,0,30,You are paying for nice settings and decent se...,if661,The original Busaba Eathai opened in 1999 in L...,"[TRY 100 - TRY 199, Vegetarian Friendly, Vegan...","106-110 Wardour Street Soho, London W1F 0TR En..."
13426,1631. Busaba Soho,0,50,Went there for lunch. Had the pad Thai and a c...,ajohnston90,The original Busaba Eathai opened in 1999 in L...,"[TRY 100 - TRY 199, Vegetarian Friendly, Vegan...","106-110 Wardour Street Soho, London W1F 0TR En..."
13427,1631. Busaba Soho,0,30,I've been here for dinner with friends a numbe...,Debby T,The original Busaba Eathai opened in 1999 in L...,"[TRY 100 - TRY 199, Vegetarian Friendly, Vegan...","106-110 Wardour Street Soho, London W1F 0TR En..."
13428,1631. Busaba Soho,0,40,Went here with my husband just after Christmas...,ruthblakeborough,The original Busaba Eathai opened in 1999 in L...,"[TRY 100 - TRY 199, Vegetarian Friendly, Vegan...","106-110 Wardour Street Soho, London W1F 0TR En..."
13429,1631. Busaba Soho,0,50,"excellent services, especially for the very fr...",W1048VMmartinac,The original Busaba Eathai opened in 1999 in L...,"[TRY 100 - TRY 199, Vegetarian Friendly, Vegan...","106-110 Wardour Street Soho, London W1F 0TR En..."


In [323]:
last_df.name.nunique()

51

### MongoDB'ye veriyi atalım

In [73]:
from pymongo import MongoClient

# This connects us to the "legistlation" database, and the "collection" [think table] news 
# in that database
client = MongoClient()
db = client.tripAdvisor_london_restaurant
collection = db.restaurant_reviews

In [355]:
# last_df=last_df.drop(columns=["info"])
last_df2 = last_df.reset_index()
data_dict = last_df2.to_dict("records")
collection.insert_many(data_dict)